In [ ]:
import datetime
import hashlib
import itertools
import json
import os
import pickle
import re
import warnings

from collections import namedtuple
from copy import copy
from pathlib import Path

In [ ]:
import cartopy.feature as cfeature
import cartopy.crs as ccrs

import joblib

import matplotlib as mpl
import matplotlib.patheffects
from matplotlib import gridspec
from matplotlib import pyplot as plt

import netCDF4
from netCDF4 import Dataset

import numpy as np

import pandas as pd

import scipy as sp

import sklearn
from sklearn.preprocessing import StandardScaler

import tqdm

In [ ]:
outlined = [
    mpl.patheffects.Stroke(linewidth=3, foreground="white"),
    mpl.patheffects.Normal(),
]

In [ ]:
TrajectoryPaths = namedtuple("TrajectoryPaths", ["date", "out", "aer", "ant", "bio", "met"])
TrajectoryDatasets = namedtuple("TrajectoryDatasets", ["date", "out", "aer", "ant", "bio", "met"])
MLDataset = namedtuple("MLDataset", ["date", "paths", "X_raw", "Y_raw", "X_train", "X_valid", "X_test", "Y_train", "Y_valid", "Y_test", "X_scaler", "Y_scaler"])

In [ ]:
OUTDIR_PATTERN = re.compile(r"(\d{4})(\d{2})(\d{2})_T(\d{2})")

In [ ]:
traj_datetimes = dict()

base = Path.cwd().parent / "trajectories"

for child in (base / "outputs" / "baseline").iterdir():
    if not child.is_dir():
        continue
    
    match = OUTDIR_PATTERN.match(child.name)
    
    if match is None:
        continue
        
    date = datetime.datetime(
        year=int(match.group(1)),
        month=int(match.group(2)),
        day=int(match.group(3)),
        hour=int(match.group(4)),
    )
    
    out_path = child / "output.nc"
    aer_path = (
        base / "inputs" / "baseline" / "HYDE_BASE_Y2018" /
        f"OUTPUT_bwd_{date.strftime('%Y%m%d')}" /
        "EMISSIONS_0422" /
        f"{date.strftime('%Y%m%d')}_7daybwd_Hyde_traj_AER_{24-date.hour:02}_L3.nc"
    )
    ant_path = (
        base / "inputs" / "baseline" / "HYDE_BASE_Y2018" /
        f"OUTPUT_bwd_{date.strftime('%Y%m%d')}" /
        "EMISSIONS_0422" /
        f"{date.strftime('%Y%m%d')}_7daybwd_Hyde_traj_ANT_{24-date.hour:02}_L3.nc"
    )
    bio_path = (
        base / "inputs" / "baseline" / "HYDE_BASE_Y2018" /
        f"OUTPUT_bwd_{date.strftime('%Y%m%d')}" /
        "EMISSIONS_0422" /
        f"{date.strftime('%Y%m%d')}_7daybwd_Hyde_traj_BIO_{24-date.hour:02}_L3.nc"
    )
    met_path = (
        base / "inputs" / "baseline" / "HYDE_BASE_Y2018" /
        f"OUTPUT_bwd_{date.strftime('%Y%m%d')}" /
        "METEO" /
        f"METEO_{date.strftime('%Y%m%d')}_R{24-date.hour:02}.nc"
    )
    
    if (
        (not out_path.exists()) or (not aer_path.exists()) or
        (not ant_path.exists()) or (not bio_path.exists()) or
        (not met_path.exists())
    ):
        raise Exception(out_path, aer_path, ant_path, bio_path, met_path)
    
    traj_datetimes[date] = TrajectoryPaths(
        date=date, out=out_path, aer=aer_path, ant=ant_path, bio=bio_path, met=met_path,
    )

traj_dates = sorted(set(d.date() for d in traj_datetimes.keys()))

In [ ]:
def load_trajectory_dataset(paths: TrajectoryPaths) -> TrajectoryDatasets:
    outds = Dataset(paths.out, "r", format="NETCDF4")
    aerds = Dataset(paths.aer, "r", format="NETCDF4")
    antds = Dataset(paths.ant, "r", format="NETCDF4")
    biods = Dataset(paths.bio, "r", format="NETCDF4")
    metds = Dataset(paths.met, "r", format="NETCDF4")
    
    return TrajectoryDatasets(
        date=paths.date, out=outds, aer=aerds, ant=antds, bio=biods, met=metds,
    )

In [ ]:
data_proj = ccrs.PlateCarree()
projection = ccrs.LambertConformal(
    central_latitude=50, central_longitude=20, standard_parallels=(25, 25)
)
extent = [-60, 60, 40, 80]

In [ ]:
def get_ccn_concentration(ds: TrajectoryDatasets):
    ccn_bin_indices, = np.nonzero(ds.out["dp_dry_fs"][:].data > 80e-9)
    ccn_concentration = np.sum(ds.out["nconc_par"][:].data[:,ccn_bin_indices,:], axis=1)
    
    return pd.DataFrame({
        "time": np.repeat(get_output_time(ds), ds.out["lev"].shape[0]),
        "level": np.tile(ds.out["lev"][:].data, ds.out["time"].shape[0]),
        "ccn": ccn_concentration.flatten(),
    }).set_index(["time", "level"])

In [ ]:
def get_output_time(ds: TrajectoryDatasets):
    fdom = datetime.datetime.strptime(
        ds.out["time"].__dict__["first_day_of_month"], "%Y-%m-%d %H:%M:%S",
    )
    dt = (ds.date - fdom).total_seconds()
    
    out_t = ds.out["time"][:].data
    
    return out_t - dt

def interpolate_meteorology_values(ds: TrajectoryDatasets, key: str):
    out_t = get_output_time(ds)
    out_h = ds.out["lev"][:].data
    
    met_t = ds.met["time"][:].data
    met_h = ds.met["lev"][:].data
    
    met_t_h = ds.met[key][:]
    
    met_t_h_int = sp.interpolate.interp2d(
        x=met_h, y=met_t, z=met_t_h, kind="linear", bounds_error=False, fill_value=0.0,
    )
    
    return met_t_h_int(x=out_h, y=out_t)

def interpolate_meteorology_time_values(ds: TrajectoryDatasets, key: str):
    out_t = get_output_time(ds)
    out_h = ds.out["lev"][:].data
    
    met_t = ds.met["time"][:].data
    
    met_t_v = ds.met[key][:]
    
    met_t_int = sp.interpolate.interp1d(
        x=met_t, y=met_t_v, kind="linear", bounds_error=False, fill_value=0.0,
    )
    
    return np.repeat(
        met_t_int(x=out_t).reshape(-1, 1),
        out_h.shape[0], axis=1,
    )

def interpolate_biogenic_emissions(ds: TrajectoryDatasets, key: str):
    out_t = get_output_time(ds)
    out_h = ds.out["lev"][:].data
    
    # depth of each box layer, assuming level heights are midpoints and end points are clamped
    out_d = (np.array(list(out_h[1:])+[out_h[-1]]) - np.array([out_h[0]]+list(out_h[:-1]))) / 2.0
    
    bio_t = ds.bio["time"][:].data
    
    # Biogenic emissions are limited to boxes at <= 10m height
    biogenic_emission_layers = np.nonzero(out_h <= 10.0)
    biogenic_emission_layer_height_cumsum = np.cumsum(out_d[biogenic_emission_layers])
    biogenic_emission_layer_proportion = biogenic_emission_layer_height_cumsum / biogenic_emission_layer_height_cumsum[-1]
    num_biogenic_emission_layers = sum(out_h <= 10.0)
    
    bio_t_h = np.zeros(shape=(out_t.size, out_h.size))
    
    bio_t_int = sp.interpolate.interp1d(
        x=bio_t, y=ds.bio[key][:], kind="linear", bounds_error=False, fill_value=0.0,
    )
    
    # Split up the biogenic emissions relative to the depth of the boxes
    bio_t_h[:,biogenic_emission_layers] = (
        np.tile(bio_t_int(x=out_t), (num_biogenic_emission_layers, 1, 1)) * biogenic_emission_layer_proportion.reshape(-1, 1, 1)
    ).T
    
    return bio_t_h

def interpolate_aerosol_emissions(ds: TrajectoryDatasets, key: str):
    out_t = get_output_time(ds)
    out_h = ds.out["lev"][:].data
    
    aer_t = ds.aer["time"][:].data
    aer_h = ds.aer["mid_layer_height"][:].data
    
    aer_t_h = ds.aer[key][:].T
    
    aer_t_h_int = sp.interpolate.interp2d(
        x=aer_h, y=aer_t, z=aer_t_h, kind="linear", bounds_error=False, fill_value=0.0,
    )
    
    return aer_t_h_int(x=out_h, y=out_t)

def interpolate_anthropogenic_emissions(ds: TrajectoryDatasets, key: str):
    out_t = get_output_time(ds)
    out_h = ds.out["lev"][:].data
    
    ant_t = ds.ant["time"][:].data
    ant_h = ds.ant["mid_layer_height"][:].data
    
    ant_t_h = ds.ant[key][:].T
    
    ant_t_h_int = sp.interpolate.interp2d(
        x=ant_h, y=ant_t, z=ant_t_h, kind="linear", bounds_error=False, fill_value=0.0,
    )
    
    return ant_t_h_int(x=out_h, y=out_t)

In [ ]:
def get_meteorology_features(ds: TrajectoryDatasets):
    return pd.DataFrame({
        "time": np.repeat(get_output_time(ds), ds.out["lev"].shape[0]),
        "level": np.tile(ds.out["lev"][:].data, ds.out["time"].shape[0]),
        "met_t": interpolate_meteorology_values(ds, "t").flatten(),
        # "met_u": interpolate_meteorology_values(ds, "u").flatten(),
        # "met_v": interpolate_meteorology_values(ds, "v").flatten(),
        "met_q": interpolate_meteorology_values(ds, "q").flatten(),
        # "met_qc": interpolate_meteorology_values(ds, "qc").flatten(),
        # "met_sp": interpolate_meteorology_time_values(ds, "sp").flatten(),
        # "met_cp": interpolate_meteorology_time_values(ds, "cp").flatten(),
        # "met_sshf": interpolate_meteorology_time_values(ds, "sshf").flatten(),
        "met_ssr": interpolate_meteorology_time_values(ds, "ssr").flatten(),
        # "met_lsp": interpolate_meteorology_time_values(ds, "lsp").flatten(),
        # "met_ewss": interpolate_meteorology_time_values(ds, "ewss").flatten(),
        # "met_nsss": interpolate_meteorology_time_values(ds, "nsss").flatten(),
        # "met_tcc": interpolate_meteorology_time_values(ds, "tcc").flatten(),
        "met_lsm": interpolate_meteorology_time_values(ds, "lsm").flatten(),
        # "met_omega": interpolate_meteorology_values(ds, "omega").flatten(),
        # "met_z": interpolate_meteorology_time_values(ds, "z").flatten(),
        # "met_mla": interpolate_meteorology_values(ds, "mla").flatten(),
        # NOTE: lp is excluded because it allows the model to overfit
        # "met_lp": interpolate_meteorology_values(ds, "lp").flatten(),
        "met_blh": interpolate_meteorology_time_values(ds, "blh").flatten(),
    }).set_index(["time", "level"])

def get_bio_emissions_features(ds: TrajectoryDatasets):
    return pd.DataFrame({
        "time": np.repeat(get_output_time(ds), ds.out["lev"].shape[0]),
        "level": np.tile(ds.out["lev"][:].data, ds.out["time"].shape[0]),
        "bio_acetaldehyde": interpolate_biogenic_emissions(ds, "acetaldehyde").flatten(),
        "bio_acetone": interpolate_biogenic_emissions(ds, "acetone").flatten(),
        "bio_butanes_and_higher_alkanes": interpolate_biogenic_emissions(ds, "butanes-and-higher-alkanes").flatten(),
        "bio_butanes_and_higher_alkenes": interpolate_biogenic_emissions(ds, "butenes-and-higher-alkenes").flatten(),
        "bio_ch4": interpolate_biogenic_emissions(ds, "CH4").flatten(),
        "bio_co": interpolate_biogenic_emissions(ds, "CO").flatten(),
        "bio_ethane": interpolate_biogenic_emissions(ds, "ethane").flatten(),
        "bio_ethanol": interpolate_biogenic_emissions(ds, "ethanol").flatten(),
        "bio_ethene": interpolate_biogenic_emissions(ds, "ethene").flatten(),
        "bio_formaldehyde": interpolate_biogenic_emissions(ds, "formaldehyde").flatten(),
        "bio_hydrogen_cyanide": interpolate_biogenic_emissions(ds, "hydrogen-cyanide").flatten(),
        "bio_iosprene": interpolate_biogenic_emissions(ds, "isoprene").flatten(),
        "bio_mbo": interpolate_biogenic_emissions(ds, "MBO").flatten(),
        "bio_methanol": interpolate_biogenic_emissions(ds, "methanol").flatten(),
        "bio_methyl_bromide": interpolate_biogenic_emissions(ds, "methyl-bromide").flatten(),
        "bio_methyl_chloride": interpolate_biogenic_emissions(ds, "methyl-chloride").flatten(),
        "bio_methyl_iodide": interpolate_biogenic_emissions(ds, "methyl-iodide").flatten(),
        "bio_other_aldehydes": interpolate_biogenic_emissions(ds, "other-aldehydes").flatten(),
        "bio_other_ketones": interpolate_biogenic_emissions(ds, "other-ketones").flatten(),
        "bio_other_monoterpenes": interpolate_biogenic_emissions(ds, "other-monoterpenes").flatten(),
        "bio_pinene_a": interpolate_biogenic_emissions(ds, "pinene-a").flatten(),
        "bio_pinene_b": interpolate_biogenic_emissions(ds, "pinene-b").flatten(),
        "bio_propane": interpolate_biogenic_emissions(ds, "propane").flatten(),
        "bio_propene": interpolate_biogenic_emissions(ds, "propene").flatten(),
        "bio_sesquiterpenes": interpolate_biogenic_emissions(ds, "sesquiterpenes").flatten(),
        "bio_toluene": interpolate_biogenic_emissions(ds, "toluene").flatten(),
        "bio_ch2br2": interpolate_biogenic_emissions(ds, "CH2Br2").flatten(),
        "bio_ch3i": interpolate_biogenic_emissions(ds, "CH3I").flatten(),
        "bio_chbr3": interpolate_biogenic_emissions(ds, "CHBr3").flatten(),
        "bio_dms": interpolate_biogenic_emissions(ds, "DMS").flatten(),
    }).set_index(["time", "level"])

def get_aer_emissions_features(ds: TrajectoryDatasets):
    return pd.DataFrame({
        "time": np.repeat(get_output_time(ds), ds.out["lev"].shape[0]),
        "level": np.tile(ds.out["lev"][:].data, ds.out["time"].shape[0]),
        "aer_3_10_nm": interpolate_aerosol_emissions(ds, "3-10nm").flatten(),
        "aer_10_20_nm": interpolate_aerosol_emissions(ds, "10-20nm").flatten(),
        "aer_20_30_nm": interpolate_aerosol_emissions(ds, "20-30nm").flatten(),
        "aer_30_50_nm": interpolate_aerosol_emissions(ds, "30-50nm").flatten(),
        "aer_50_70_nm": interpolate_aerosol_emissions(ds, "50-70nm").flatten(),
        "aer_70_100_nm": interpolate_aerosol_emissions(ds, "70-100nm").flatten(),
        "aer_100_200_nm": interpolate_aerosol_emissions(ds, "100-200nm").flatten(),
        "aer_200_400_nm": interpolate_aerosol_emissions(ds, "200-400nm").flatten(),
        "aer_400_1000_nm": interpolate_aerosol_emissions(ds, "400-1000nm").flatten(),
    }).set_index(["time", "level"])

def get_ant_emissions_features(ds: TrajectoryDatasets):
    return pd.DataFrame({
        "time": np.repeat(get_output_time(ds), ds.out["lev"].shape[0]),
        "level": np.tile(ds.out["lev"][:].data, ds.out["time"].shape[0]),
        "ant_co": interpolate_anthropogenic_emissions(ds, "co").flatten(),
        "ant_nox": interpolate_anthropogenic_emissions(ds, "nox").flatten(),
        "ant_co2": interpolate_anthropogenic_emissions(ds, "co2").flatten(),
        "ant_nh3": interpolate_anthropogenic_emissions(ds, "nh3").flatten(),
        "ant_ch4": interpolate_anthropogenic_emissions(ds, "ch4").flatten(),
        "ant_so2": interpolate_anthropogenic_emissions(ds, "so2").flatten(),
        "ant_nmvoc": interpolate_anthropogenic_emissions(ds, "nmvoc").flatten(),
        "ant_alcohols": interpolate_anthropogenic_emissions(ds, "alcohols").flatten(),
        "ant_ethane": interpolate_anthropogenic_emissions(ds, "ethane").flatten(),
        "ant_propane": interpolate_anthropogenic_emissions(ds, "propane").flatten(),
        "ant_butanes": interpolate_anthropogenic_emissions(ds, "butanes").flatten(),
        "ant_pentanes": interpolate_anthropogenic_emissions(ds, "pentanes").flatten(),
        "ant_hexanes": interpolate_anthropogenic_emissions(ds, "hexanes").flatten(),
        "ant_ethene": interpolate_anthropogenic_emissions(ds, "ethene").flatten(),
        "ant_propene": interpolate_anthropogenic_emissions(ds, "propene").flatten(),
        "ant_acetylene": interpolate_anthropogenic_emissions(ds, "acetylene").flatten(),
        "ant_isoprene": interpolate_anthropogenic_emissions(ds, "isoprene").flatten(),
        "ant_monoterpenes": interpolate_anthropogenic_emissions(ds, "monoterpenes").flatten(),
        "ant_other_alkenes_and_alkynes": interpolate_anthropogenic_emissions(ds, "other-alkenes-and-alkynes").flatten(),
        "ant_benzene": interpolate_anthropogenic_emissions(ds, "benzene").flatten(),
        "ant_toluene": interpolate_anthropogenic_emissions(ds, "toluene").flatten(),
        "ant_xylene": interpolate_anthropogenic_emissions(ds, "xylene").flatten(),
        "ant_trimethylbenzene": interpolate_anthropogenic_emissions(ds, "trimethylbenzene").flatten(),
        "ant_other_aromatics": interpolate_anthropogenic_emissions(ds, "other-aromatics").flatten(),
        "ant_esters": interpolate_anthropogenic_emissions(ds, "esters").flatten(),
        "ant_ethers": interpolate_anthropogenic_emissions(ds, "ethers").flatten(),
        "ant_formaldehyde": interpolate_anthropogenic_emissions(ds, "formaldehyde").flatten(),
        "ant_other_aldehydes": interpolate_anthropogenic_emissions(ds, "other-aldehydes").flatten(),
        "ant_total_ketones": interpolate_anthropogenic_emissions(ds, "total-ketones").flatten(),
        "ant_total_acids": interpolate_anthropogenic_emissions(ds, "total-acids").flatten(),
        "ant_other_vocs": interpolate_anthropogenic_emissions(ds, "other-VOCs").flatten(),
    }).set_index(["time", "level"])

In [ ]:
# https://stackoverflow.com/a/67809235
def df_to_numpy(df):
    try:
        shape = [len(level) for level in df.index.levels]
    except AttributeError:
        shape = [len(df.index)]
    ncol = df.shape[-1]
    if ncol > 1:
        shape.append(ncol)
    return df.to_numpy().reshape(shape)

In [ ]:
def generate_time_level_windows():
    # -0.5h, -1.5h, -3h, -6h, -12h, -24h, -48h
    # 0, -2, -5, -11, -23, -47, -95
    time_windows = [(0, 0), (-2, -1), (-5, -3), (-11, -6), (-23, -12), (-47, -24), (-95, -48)]
    
    # +1l, +2l, +4l, +8l, +16l, +32l, +64
    top_windows = [(1, 1), (1, 2), (1, 4), (2, 8), (2, 16), (3, 32), (3, 64)]
    mid_windows = [(0, 0), (0, 0), (0, 0), (-1, 1), (-1, 1), (-2, 2), (-2, 2)]
    bot_windows = [(-1, -1), (-2, -1), (-4, -1), (-8, -2), (-16, -2), (-32, -3), (-64, -3)]
    
    return list(itertools.chain(
        zip(time_windows, top_windows), zip(time_windows, mid_windows), zip(time_windows, bot_windows),
    ))

In [ ]:
def generate_windowed_feature_names(columns):
    time_windows = ["-0.5h", "-1.5h", "-3h", "-6h", "-12h", "-24h", "-48h"]
    
    top_windows = ["+1l", "+2l", "+4l", "+8l", "+16l", "+32l", "+64l"]
    mid_windows = ["+0l", "+0l", "+0l", "±1l", "±1l", "±2l", "±2l"]
    bot_windows = ["-1l", "-2l", "-4l", "-8l", "-16l", "-32l", "-64l"]
    
    names = []
    
    for (t, l) in itertools.chain(
        zip(time_windows, top_windows), zip(time_windows, mid_windows), zip(time_windows, bot_windows),
    ):
        for c in columns:
            names.append(f"{c}{t}{l}")
    
    return names

In [ ]:
def time_level_window_mean_v1(input, t_range, l_range):
    output = np.zeros(shape=input.shape)

    for t in range(input.shape[0]):
        for l in range(input.shape[1]):
            for f in range(input.shape[2]):
                window = input[
                    min(max(0, t+t_range[0]), input.shape[0]):max(0, min(t+1+t_range[1], input.shape[0])),
                    min(max(0, l+l_range[0]), input.shape[1]):max(0, min(l+1+l_range[1], input.shape[1])),
                    f
                ]

                output[t,l,f] = np.mean(window) if window.size > 0 else 0.0
    
    return output

def time_level_window_mean_v2(input, t_range, l_range):
    output = np.zeros(shape=input.shape)

    for t in range(input.shape[0]):
        mint = min(max(0, t+t_range[0]), input.shape[0])
        maxt = max(0, min(t+1+t_range[1], input.shape[0]))
        
        if mint == maxt:
            continue
        
        for l in range(input.shape[1]):
            minl = min(max(0, l+l_range[0]), input.shape[1])
            maxl = max(0, min(l+1+l_range[1], input.shape[1]))
            
            if minl == maxl:
                continue
                
            output[t,l,:] = np.mean(input[mint:maxt,minl:maxl,:], axis=(0,1))
    
    return output

def time_level_window_mean_v3(input, t_range, l_range):
    min_t = min(t_range[0], 0)
    max_t = max(0, t_range[1])
    abs_t = max(abs(min_t), abs(max_t))
    
    min_l = min(l_range[0], 0)
    max_l = max(0, l_range[1])
    abs_l = max(abs(min_l), abs(max_l))
    
    kernel = np.zeros(shape=(abs_t*2 + 1, abs_l*2 + 1, 1))
    kernel[t_range[0]+abs_t:t_range[1]+abs_t+1,l_range[0]+abs_l:l_range[1]+abs_l+1,:] = 1.0
    kernel = kernel[::-1,::-1]
    
    quot = sp.ndimage.convolve(np.ones_like(input), kernel, mode='constant', cval=0.0)
    
    result = np.zeros_like(input)
    
    np.divide(
        sp.ndimage.convolve(input, kernel, mode='constant', cval=0.0),
        quot, out=result, where=quot > 0,
    )
    
    return result

In [ ]:
def get_raw_features_for_dataset(ds: TrajectoryDatasets):
    bio_features = get_bio_emissions_features(ds)
    aer_features = get_aer_emissions_features(ds) * 1e21
    ant_features = get_ant_emissions_features(ds)
    met_features = get_meteorology_features(ds)
    
    return pd.concat([
        bio_features, aer_features, ant_features, met_features,
    ], axis="columns")

In [ ]:
def get_features_from_raw_features(raw_features):
    raw_features_np = df_to_numpy(raw_features)
    
    features_np = np.concatenate([
        raw_features.index.get_level_values(0).to_numpy().reshape(
            (raw_features.index.levels[0].size, raw_features.index.levels[1].size, 1)
        ),
        raw_features.index.get_level_values(1).to_numpy().reshape(
            (raw_features.index.levels[0].size, raw_features.index.levels[1].size, 1)
        )
    ] + joblib.Parallel(n_jobs=-1)([
        joblib.delayed(time_level_window_mean_v2)(raw_features_np, t, l) for t, l in generate_time_level_windows()
    ]), axis=2)
    
    # Trim off the first two days, for which the time features are ill-defined
    features_np_trimmed = features_np[95:-1,:,:]
    
    feature_names = ["time", "level"] + generate_windowed_feature_names(raw_features.columns)
    
    features = pd.DataFrame(features_np_trimmed.reshape(
        features_np_trimmed.shape[0]*features_np_trimmed.shape[1], features_np_trimmed.shape[2],
    ), columns=feature_names).set_index(["time", "level"])
    
    return features

In [ ]:
def get_labels_for_dataset(ds: TrajectoryDatasets):
    ccn_concentration = get_ccn_concentration(ds)
    
    ccn_concentration_np = df_to_numpy(ccn_concentration)
    
    labels_np = np.concatenate([
        ccn_concentration.index.get_level_values(0).to_numpy().reshape(
            (ccn_concentration.index.levels[0].size, ccn_concentration.index.levels[1].size, 1)
        ),
        ccn_concentration.index.get_level_values(1).to_numpy().reshape(
            (ccn_concentration.index.levels[0].size, ccn_concentration.index.levels[1].size, 1)
        ),
        ccn_concentration_np.reshape(
            (ccn_concentration_np.shape[0], ccn_concentration_np.shape[1], 1)
        ),
    ], axis=2)
    
    # Trim off the first two days, for which the time features are ill-defined
    labels_np_trimmed = labels_np[96:,:,:]
    
    label_names = ["time", "level", "ccn"]
    
    labels = pd.DataFrame(labels_np_trimmed.reshape(
        labels_np_trimmed.shape[0]*labels_np_trimmed.shape[1], labels_np_trimmed.shape[2],
    ), columns=label_names).set_index(["time", "level"])
    
    return labels

In [ ]:
def hash_for_dt(dt):
    if not(isinstance(dt, tuple) or isinstance(dt, list)):
        dt = [dt]
    
    dt_str = '.'.join(dtt.strftime('%d.%m.%Y-%H:00%z') for dtt in dt)
    
    h = hashlib.shake_256()
    h.update(dt_str.encode('ascii'))
    
    return h

In [ ]:
"""
Clumped 0/1 sampler using a Markov Process

P(0) = p and P(1) = 1-p
clump = 0 => IID samples
clump -> 1 => highly correlated samples

"""
class Clump:
    def __init__(self, p=0.5, clump=0.0, rng=None):
        a = 1 - (1-p)*(1-clump)
        b = (1-a)*p/(1-p)
        
        self.C = np.array([[a, 1-a],[b, 1-b]])
        
        self.i = 0 if rng.random() < p else 1
    
    def sample(self, rng):
        p = self.C[self.i,0]
        u = rng.random()
        
        self.i = 0 if u < p else 1
        
        return self.i
    
    def steady(self, X):
        return np.matmul(X, self.C)

In [ ]:
def train_test_split(X, Y, test_size=0.25, random_state=None, shuffle=True, clump=0.0):
    assert len(X) == len(Y)
    assert type(X) == type(Y)
    assert test_size > 0.0
    assert test_size < 1.0
    assert random_state is not None
    assert clump >= 0.0
    assert clump < 1.0
    
    c = Clump(p=test_size, clump=clump, rng=random_state)
    
    if isinstance(X, pd.DataFrame):
        assert X.index.values.shape == Y.index.values.shape
        
        # Split only based on the first-level index instead of flattening
        n1 = len(X.index.levels[1])
        n0 = len(X) // n1
        
        C = np.array([c.sample(random_state) for _ in range(n0)])
        I_train, = np.nonzero(C)
        I_train = np.repeat(I_train, n1) * n1 + np.tile(np.arange(n1), len(I_train))
        I_test, = np.nonzero(1-C)
        I_test = np.repeat(I_test, n1) * n1 + np.tile(np.arange(n1), len(I_test))
    else:
        C = np.array([c.sample(random_state) for _ in range(len(X))])
        I_train, = np.nonzero(C)
        I_test, = np.nonzero(1-C)
    
    if shuffle:
        random_state.shuffle(I_train)
        random_state.shuffle(I_test)
    
    if isinstance(X, pd.DataFrame):
        X_train = X.iloc[I_train]
        X_test = X.iloc[I_test]
        
        Y_train = Y.iloc[I_train]
        Y_test = Y.iloc[I_test]
    else:
        X_train = X[I_train]
        X_test = X[I_test]
        
        Y_train = Y[I_train]
        Y_test = Y[I_test]
    
    return X_train, X_test, Y_train, Y_test

In [ ]:
def load_and_cache_dataset(dt: datetime.datetime, clump: float, datasets: dict) -> MLDataset:
    if isinstance(dt, tuple) or isinstance(dt, list):
        dt = tuple(sorted(dt))
    
    cached = datasets.get((dt, clump))
    
    if cached is not None:
        return cached
    
    if isinstance(dt, tuple) or isinstance(dt, list):
        mls = [load_and_cache_dataset(dtt, clump, datasets) for dtt in dt]

        dp = tuple(ml.paths for ml in mls)
        X_raw = pd.concat([ml.X_raw for ml in mls], axis='index')
        Y = pd.concat([ml.Y_raw for ml in mls], axis='index')

        train_features = np.concatenate([ml.X_scaler.inverse_transform(ml.X_train) for ml in mls], axis=0)
        train_labels = np.concatenate([ml.Y_scaler.inverse_transform(ml.Y_train) for ml in mls], axis=0)
        valid_features = np.concatenate([ml.X_scaler.inverse_transform(ml.X_valid) for ml in mls], axis=0)
        valid_labels = np.concatenate([ml.Y_scaler.inverse_transform(ml.Y_valid) for ml in mls], axis=0)
        test_features = np.concatenate([ml.X_scaler.inverse_transform(ml.X_test) for ml in mls], axis=0)
        test_labels = np.concatenate([ml.Y_scaler.inverse_transform(ml.Y_test) for ml in mls], axis=0)
    else:
        dp = traj_datetimes[dt]
        ds = load_trajectory_dataset(dp)

        X_raw = get_raw_features_for_dataset(ds)

        X = get_features_from_raw_features(X_raw)
        Y = np.log10(get_labels_for_dataset(ds) + 1)

        rng = np.random.RandomState(seed=int.from_bytes(hash_for_dt(dt).digest(4), 'little'))

        train_features, test_features, train_labels, test_labels = train_test_split(
            X, Y, test_size=0.25, random_state=rng, clump=clump,
        )
        train_features, valid_features, train_labels, valid_labels = train_test_split(
            train_features, train_labels, test_size=1.0/3.0, random_state=rng, clump=clump,
        )

        # Close the NetCDF dataset
        ds.out.close()

    # Scale features to N(0,1)
    # - only fit on training data
    # - OOD inputs for constants at training time are blown up
    feature_scaler = StandardScaler().fit(train_features)
    feature_scaler.scale_[np.nonzero(feature_scaler.var_ == 0.0)] = np.nan_to_num(np.inf)

    label_scaler = StandardScaler().fit(train_labels)

    train_features = feature_scaler.transform(train_features)
    train_labels = label_scaler.transform(train_labels)
    valid_features = feature_scaler.transform(valid_features)
    valid_labels = label_scaler.transform(valid_labels)
    test_features = feature_scaler.transform(test_features)
    test_labels = label_scaler.transform(test_labels)

    dataset = MLDataset(
        date=dt, paths=dp, X_raw=X_raw, Y_raw=Y,
        X_train=train_features, X_valid=valid_features, X_test=test_features,
        Y_train=train_labels, Y_valid=valid_labels, Y_test=test_labels,
        X_scaler=feature_scaler, Y_scaler=label_scaler,
    )

    datasets[(dt, clump)] = dataset
    
    return dataset

In [ ]:
DATASETS = dict()

In [ ]:
import abc

class OutOfDistributionDetector(abc.ABC):
    @abc.abstractmethod
    def fit(self, X_id, Y_id, X_id_valid, Y_id_valid):
        return self
    
    @abc.abstractmethod
    def score(self, X_test):
        return None
    
    @classmethod
    def name(cls):
        return cls.__name__

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel

class GaussianProcessPercentileDetector(OutOfDistributionDetector):
    def __init__(self, *args, rng=None, **kwargs):
        self.model = GaussianProcessRegressor(*args, kernel=(
            RBF() + WhiteKernel()
        ), random_state=rng, **kwargs)
        self.rng = rng
    
    def fit(self, X_id, Y_id, X_id_valid, Y_id_valid):
        num_X = len(X_id)
        
        # Safety hatch to limit computing time
        if num_X >= 10_000:
            I_train = self.rng.choice(len(X_id), size=num_X//6)
            I_valid = np.ones(len(X_id))
            I_valid[I_train] = 0
            I_valid, = np.nonzero(I_valid)
            
            X_id_valid = np.concatenate([X_id_valid, X_id[I_valid]], axis=0)
            Y_id_valid = np.concatenate([Y_id_valid, Y_id[I_valid]], axis=0)
            
            X_id = X_id[I_train]
            Y_id = Y_id[I_train]
        
        self.model.fit(X_id, Y_id)
        
        self.uq_valid = np.sort(self.model.predict(X_id_valid, return_std=True)[1])
        
        return self
    
    def score(self, X_test):
        return 1.0 - np.searchsorted(
            self.uq_valid, self.model.predict(X_test, return_std=True)[1],
        ) / len(self.uq_valid)

In [ ]:
from sklearn.covariance import EmpiricalCovariance
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPRegressor

class AutoAssociativeMahalanobisPercentileDetector(OutOfDistributionDetector):
    def __init__(self, *args, rng=None, **kwargs):
        self.rng = rng
        
    def fit(self, X_id, Y_id, X_id_valid, Y_id_valid):
        print("Fitting PCA")
        
        bn = np.searchsorted(np.cumsum(
            PCA(random_state=self.rng).fit(X_id).explained_variance_ratio_
        ), 0.95)
        
        print(f"Fitting AA with bn={bn}")
        
        self.model = MLPRegressor(
            hidden_layer_sizes=[X_id.shape[1]*2, bn, X_id.shape[1]*2],
            activation="relu", solver="adam", random_state=self.rng,
        ).fit(X_id, X_id)
        
        print("Fitting covariance")
        
        self.cov = EmpiricalCovariance().fit((self.model.predict(X_id) - X_id))
        
        self.err_valid = np.sort(self.cov.mahalanobis(
            self.model.predict(X_id_valid) - X_id_valid
        ))
        
        return self
    
    def score(self, X_test):
        return 1.0 - np.searchsorted(
            self.err_valid, self.cov.mahalanobis((self.model.predict(X_test) - X_test)),
        ) / len(self.err_valid)

In [ ]:
import tensorflow as tf

from sklearn.neural_network import MLPClassifier

class LogisticClassifierDetector(OutOfDistributionDetector):
    def __init__(self, *args, rng=None, **kwargs):
        self.rng = rng
        
    def fit(self, X_id, Y_id, X_id_valid, Y_id_valid):
        print("Fitting PCA")
        
        bn = np.searchsorted(np.cumsum(
            PCA(random_state=self.rng).fit(X_id).explained_variance_ratio_
        ), 0.95)
        
        print(f"Fitting AA with bn={bn}")
        
        tf.random.set_seed(int.from_bytes(self.rng.bytes(4), "little"))
        
        self.aa_model = tf.keras.Sequential(
            [
                tf.keras.layers.InputLayer(input_shape=X_id.shape[1:]),
                tf.keras.layers.Dense(X_id.shape[1]*2, activation="relu", name="aa-enc"),
                tf.keras.layers.Dense(bn, activation="relu", name="aa-bn"),
                tf.keras.layers.Dense(X_id.shape[1]*2, activation="relu", name="aa-dec"),
                tf.keras.layers.Dense(X_id.shape[1], name="aa-X"),
            ]
        )
        self.aa_model.compile(optimizer='adam', loss="mse", metrics=["mse", "mae"])
        self.aa_model.fit(
            X_id, X_id,
            validation_data=(X_id_valid, X_id_valid),
            batch_size=200, epochs=200, verbose=1, callbacks=[
                tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=10,
                    restore_best_weights=True,
                )
            ],
        )
        
        print("Generate FGSM inputs")
        
        x_id = tf.constant(X_id_valid, dtype=tf.float32)
        x_id = tf.random.normal(mean=x_id, stddev=0.01, shape=x_id.shape)
        
        with tf.GradientTape() as tape:
            tape.watch(x_id)

            x_pred_id = self.aa_model(x_id)
            x_mse_id = (x_pred_id - x_id) ** 2

        adv_grad = tape.gradient(x_mse_id, x_id)

        X_ood = (x_id + tf.math.sign(adv_grad) * tf.math.abs(tf.random.normal(
            mean=2.0, stddev=0.5, shape=[len(X_id_valid), 1],
        ))).numpy()
        
        print("Train classifier")
        
        self.model = tf.keras.Sequential(
            [
                tf.keras.layers.InputLayer(input_shape=X_id.shape[1:]),
                tf.keras.layers.Dense(bn, activation="relu", name="io-bn"),
                tf.keras.layers.Dense(1, activation="sigmoid", name="id-ood-sigmoid"),
            ]
        )
        self.model.compile(optimizer='adam', loss="binary_crossentropy")
        self.model.fit(
            np.concatenate([X_id_valid, X_ood], axis=0),
            np.concatenate([np.ones(len(X_id_valid)), np.zeros(len(X_ood))], axis=0),
            validation_split=0.1,
            batch_size=200, epochs=200, verbose=1, callbacks=[
                tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=10,
                    restore_best_weights=True,
                )
            ],
        )
        
        return self
    
    def score(self, X_test):
        return self.model.predict(X_test)

In [ ]:
class TPokeNoise(tf.keras.layers.Layer):
    def __init__(self, t, **kwargs):
        super().__init__(**kwargs)
        
        self.t = t

    def call(self, inputs, training=None):
        return tf.math.abs(tf.random.normal(
            mean=self.t, stddev=0.1, shape=(tf.shape(inputs)[0], 1),
        ))

class TPokeClassifierModel(tf.keras.Model):
    def __init__(self, indim, bn, aa_model):
        input = tf.keras.Input(shape=[indim])
        
        dense_1 = tf.keras.layers.Dense(bn, activation="relu", name="dense-1")(input)
        is_id = tf.keras.layers.Dense(1, activation="sigmoid", name="is-id")(dense_1)
        
        super().__init__(input, is_id)
        
        self.aa_model = aa_model
        self.noise = tf.keras.layers.GaussianNoise(0.01)
        
    def compile(self, *args, **kwargs):
        self.t = self.add_weight(initializer="ones", trainable=False, name="t")
        self.t.assign(self.t * 2.0)
        
        super().compile(*args, **kwargs)
    
    def train_step(self, data):
        x_id = data
        
        x_ood = self.noise(x_id)
        
        with tf.GradientTape() as tape:
            tape.watch(x_ood)

            x_pred_ood = self.aa_model(x_ood, training=False)
            x_mse_ood = (x_pred_ood - x_ood) ** 2

        adv_grad = tape.gradient(x_mse_ood, x_ood)
        x_ood += tf.stop_gradient(tf.math.sign(adv_grad)) * TPokeNoise(self.t)(x_ood)
        
        with tf.GradientTape() as tape:
            c_id = self.call(x_id)
            c_ood = self.call(x_ood)
            
            loss = self.compiled_loss(c_id, c_ood)
            
        grads = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
        
        self.t.assign(self.t * (1.0 - 0.001 * tf.math.minimum(
            tf.math.sign(tf.math.reduce_mean(c_id - 0.95)),
            tf.math.sign(tf.math.reduce_mean(0.05 - c_ood)),
        )))
        
        return {m.name: m.result() for m in self.metrics}
    
    def test_step(self, data):
        x_id = data
        
        x_ood = self.noise(x_id)
        
        with tf.GradientTape() as tape:
            tape.watch(x_ood)

            x_pred_ood = self.aa_model(x_ood, training=False)
            x_mse_ood = (x_pred_ood - x_ood) ** 2

        adv_grad = tape.gradient(x_mse_ood, x_ood)
        x_ood += tf.stop_gradient(tf.math.sign(adv_grad)) * TPokeNoise(self.t)(x_ood)
        
        c_id = self.call(x_id)
        c_ood = self.call(x_ood)

        loss = self.compiled_loss(c_id, c_ood)
        
        return {m.name: m.result() for m in self.metrics}

    
def id_ood_loss(c_id, c_ood):
    return -(tf.math.log(tf.maximum(c_id, 1e-6)) + tf.math.log(tf.maximum(1.0 - c_ood, 1e-6)))

tf.keras.utils.get_custom_objects()["id_ood_loss"] = id_ood_loss

class LogisticTPokeClassifierDetector(OutOfDistributionDetector):
    def __init__(self, *args, rng=None, **kwargs):
        self.rng = rng
        
    def fit(self, X_id, Y_id, X_id_valid, Y_id_valid):
        print("Fitting PCA")
        
        bn = np.searchsorted(np.cumsum(
            PCA(random_state=self.rng).fit(X_id).explained_variance_ratio_
        ), 0.95)
        
        print(f"Fitting AA with bn={bn}")
        
        tf.random.set_seed(int.from_bytes(self.rng.bytes(4), "little"))
        
        self.aa_model = tf.keras.Sequential(
            [
                tf.keras.layers.InputLayer(input_shape=X_id.shape[1:]),
                tf.keras.layers.Dense(X_id.shape[1]*2, activation="relu", name="aa-enc"),
                tf.keras.layers.Dense(bn, activation="relu", name="aa-bn"),
                tf.keras.layers.Dense(X_id.shape[1]*2, activation="relu", name="aa-dec"),
                tf.keras.layers.Dense(X_id.shape[1], name="aa-X"),
            ]
        )
        self.aa_model.compile(optimizer='adam', loss="mse", metrics=["mse", "mae"])
        self.aa_model.fit(
            X_id, X_id,
            validation_data=(X_id_valid, X_id_valid),
            batch_size=200, epochs=200, verbose=1, callbacks=[
                tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=10,
                    restore_best_weights=True,
                )
            ],
        )
        
        print("Train classifier")
        
        self.model = TPokeClassifierModel(X_id.shape[1], bn, self.aa_model)
        self.model.compile(optimizer='adam', loss=id_ood_loss)
        self.model.fit(X_id_valid, validation_split=0.1,
            batch_size=200, epochs=200, verbose=1, callbacks=[
                tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=25,
                    restore_best_weights=True,
                )
            ],
        )
        
        print(f"T-poking resulted in t={self.model.t}")
        
        return self
    
    def score(self, X_test):
        return self.model.predict(X_test)

In [ ]:
from sklearn.neighbors import KernelDensity

class GaussianProcessIDWeightDetector(OutOfDistributionDetector):
    def __init__(self, *args, rng=None, **kwargs):
        self.gp_model = GaussianProcessRegressor(*args, kernel=(
            RBF() + WhiteKernel()
        ), random_state=rng, **kwargs)
        self.rng = rng
        
    def fit(self, X_id, Y_id, X_id_valid, Y_id_valid):
        print("Fitting Gaussian Process")
        
        num_X = len(X_id)
        
        # Safety hatch to limit computing time
        if num_X >= 10_000:
            I_train = self.rng.choice(len(X_id), size=num_X//6)
            I_valid = np.ones(len(X_id))
            I_valid[I_train] = 0
            I_valid, = np.nonzero(I_valid)
            
            X_id_valid = np.concatenate([X_id_valid, X_id[I_valid]], axis=0)
            Y_id_valid = np.concatenate([Y_id_valid, Y_id[I_valid]], axis=0)
            
            X_id = X_id[I_train]
            Y_id = Y_id[I_train]
        
        self.gp_model.fit(X_id, Y_id)
        
        print("Fitting id_err")
        
        id_err = self.gp_model.predict(X_id_valid, return_std=True)[1]
        self.err_scale = np.mean(id_err)
        id_err /= self.err_scale
        self.id_err = KernelDensity(bandwidth=0.1, kernel="linear").fit(id_err.reshape(-1, 1))
        
        print("Fitting all_err")
        
        X_all = self.rng.normal(loc=0.0, scale=1.0, size=X_id_valid.shape)
        all_err = self.gp_model.predict(X_all, return_std=True)[1] / self.err_scale
        self.all_err = KernelDensity(bandwidth=0.1, kernel="linear").fit(all_err.reshape(-1, 1))
        
        print("Fitting id_scale")
        
        all_as_id = np.exp(self.id_err.score_samples(all_err.reshape(-1, 1)))
        all_as_all = np.exp(self.all_err.score_samples(all_err.reshape(-1, 1)))
        
        self.id_scale = np.nan_to_num(np.nanmin(all_as_all / all_as_id), nan=1.0, posinf=1.0, neginf=1.0)
        
        return self
    
    def score(self, X_test):
        print("Calculating test_err")
        
        test_err = self.gp_model.predict(X_test, return_std=True)[1] / self.err_scale
        
        print("Calculating test_as_id and test_as_all")
        
        test_as_id = np.exp(self.id_err.score_samples(test_err.reshape(-1, 1)))
        test_as_all = np.exp(self.all_err.score_samples(test_err.reshape(-1, 1)))
        
        return np.nan_to_num(test_as_id / np.maximum(test_as_id, test_as_all), nan=0.0)

In [ ]:
from sklearn.neighbors import KernelDensity

class AutoAssociativeMahalanobisIDWeightDetector(OutOfDistributionDetector):
    def __init__(self, *args, rng=None, **kwargs):
        self.rng = rng
        
    def fit(self, X_id, Y_id, X_id_valid, Y_id_valid):
        print("Fitting PCA")
        
        bn = np.searchsorted(np.cumsum(
            PCA(random_state=self.rng).fit(X_id).explained_variance_ratio_
        ), 0.95)
        
        print(f"Fitting AA with bn={bn}")
        
        self.model = MLPRegressor(
            hidden_layer_sizes=[X_id.shape[1]*2, bn, X_id.shape[1]*2],
            activation="relu", solver="adam", random_state=self.rng,
        ).fit(X_id, X_id)
        
        print("Fitting covariance")
        
        self.cov = EmpiricalCovariance().fit((self.model.predict(X_id) - X_id))
        
        print("Fitting id_err")
        
        id_err = self.cov.mahalanobis((self.model.predict(X_id_valid) - X_id_valid))
        self.err_scale = np.mean(id_err)
        id_err /= self.err_scale
        self.id_err = KernelDensity(bandwidth=0.1, kernel="linear").fit(id_err.reshape(-1, 1))
        
        print("Fitting all_err")
        
        X_all = self.rng.normal(loc=0.0, scale=1.0, size=X_id_valid.shape)
        all_err = self.cov.mahalanobis((self.model.predict(X_all) - X_all)) / self.err_scale
        self.all_err = KernelDensity(bandwidth=0.1, kernel="linear").fit(all_err.reshape(-1, 1))
        
        print("Fitting id_scale")
        
        all_as_id = np.exp(self.id_err.score_samples(all_err.reshape(-1, 1)))
        all_as_all = np.exp(self.all_err.score_samples(all_err.reshape(-1, 1)))
        
        self.id_scale = np.nan_to_num(np.nanmin(all_as_all / all_as_id), nan=1.0, posinf=1.0, neginf=1.0)
        
        return self
    
    def score(self, X_test):
        print("Calculating test_err")
        
        test_err = self.cov.mahalanobis((self.model.predict(X_test) - X_test)) / self.err_scale
        
        print("Calculating test_as_id and test_as_all")
        
        test_as_id = np.exp(self.id_err.score_samples(test_err.reshape(-1, 1)))
        test_as_all = np.exp(self.all_err.score_samples(test_err.reshape(-1, 1)))
        
        return np.nan_to_num(test_as_id / np.maximum(test_as_id, test_as_all), nan=0.0)

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel

class GaussianProcessLogisticDetector(OutOfDistributionDetector):
    def __init__(self, *args, rng=None, **kwargs):
        self.gp = GaussianProcessRegressor(*args, kernel=(
            RBF() + WhiteKernel()
        ), random_state=rng, **kwargs)
        self.rng = rng
    
    def fit(self, X_id, Y_id, X_id_valid, Y_id_valid):
        print("Fitting PCA")
        
        bn = np.searchsorted(np.cumsum(
            PCA(random_state=self.rng).fit(X_id).explained_variance_ratio_
        ), 0.95)
        
        print(f"Fitting AA with bn={bn}")
        
        tf.random.set_seed(int.from_bytes(self.rng.bytes(4), "little"))
        
        self.aa_model = tf.keras.Sequential(
            [
                tf.keras.layers.InputLayer(input_shape=X_id.shape[1:]),
                tf.keras.layers.Dense(X_id.shape[1]*2, activation="relu", name="aa-enc"),
                tf.keras.layers.Dense(bn, activation="relu", name="aa-bn"),
                tf.keras.layers.Dense(X_id.shape[1]*2, activation="relu", name="aa-dec"),
                tf.keras.layers.Dense(X_id.shape[1], name="aa-X"),
            ]
        )
        self.aa_model.compile(optimizer='adam', loss="mse", metrics=["mse", "mae"])
        self.aa_model.fit(
            X_id, X_id,
            validation_data=(X_id_valid, X_id_valid),
            batch_size=200, epochs=200, verbose=1, callbacks=[
                tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=10,
                    restore_best_weights=True,
                )
            ],
        )
        
        print("Generate FGSM inputs")
        
        x_id = tf.constant(X_id_valid, dtype=tf.float32)
        x_id = tf.random.normal(mean=x_id, stddev=0.01, shape=x_id.shape)
        
        with tf.GradientTape() as tape:
            tape.watch(x_id)

            x_pred_id = self.aa_model(x_id)
            x_mse_id = (x_pred_id - x_id) ** 2

        adv_grad = tape.gradient(x_mse_id, x_id)

        X_ood = (x_id + tf.math.sign(adv_grad) * tf.math.abs(tf.random.normal(
            mean=2.0, stddev=0.5, shape=[len(X_id_valid), 1],
        ))).numpy()
        
        print("Fitting Gaussian Process")
        
        num_X = len(X_id)
        
        # Safety hatch to limit computing time
        if num_X >= 10_000:
            I_train = self.rng.choice(len(X_id), size=num_X//6)
            I_valid = np.ones(len(X_id))
            I_valid[I_train] = 0
            I_valid, = np.nonzero(I_valid)
            
            X_id_valid = np.concatenate([X_id_valid, X_id[I_valid]], axis=0)
            Y_id_valid = np.concatenate([Y_id_valid, Y_id[I_valid]], axis=0)
            
            X_id = X_id[I_train]
            Y_id = Y_id[I_train]
        
        self.gp.fit(X_id, Y_id)
        
        print("Train classifier")
        
        U_id = self.gp.predict(X_id_valid, return_std=True)[1]
        U_ood = self.gp.predict(X_ood, return_std=True)[1]
        
        self.scaler = StandardScaler().fit(U_id.reshape(-1, 1))
        
        self.model = LogisticRegression(
            penalty='none', class_weight="balanced", random_state=self.rng,
        ).fit(
            np.concatenate([
                self.scaler.transform(U_id.reshape(-1, 1)),
                self.scaler.transform(U_ood.reshape(-1, 1)),
            ], axis=0).reshape(-1, 1),
            np.concatenate([
                np.ones(len(U_id)), np.zeros(len(U_ood)),
            ], axis=0),
        )
        
        return self
    
    def score(self, X_test):
        return self.model.predict_proba(self.scaler.transform(
            self.gp.predict(X_test, return_std=True)[1].reshape(-1, 1)
        ))[:,1]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

class AutoAssociativeMahalanobisLogisticDetector(OutOfDistributionDetector):
    def __init__(self, *args, rng=None, **kwargs):
        self.rng = rng
        
    def fit(self, X_id, Y_id, X_id_valid, Y_id_valid):
        print("Fitting PCA")
        
        bn = np.searchsorted(np.cumsum(
            PCA(random_state=self.rng).fit(X_id).explained_variance_ratio_
        ), 0.95)
        
        print(f"Fitting AA with bn={bn}")
        
        tf.random.set_seed(int.from_bytes(self.rng.bytes(4), "little"))
        
        self.aa_model = tf.keras.Sequential(
            [
                tf.keras.layers.InputLayer(input_shape=X_id.shape[1:]),
                tf.keras.layers.Dense(X_id.shape[1]*2, activation="relu", name="aa-enc"),
                tf.keras.layers.Dense(bn, activation="relu", name="aa-bn"),
                tf.keras.layers.Dense(X_id.shape[1]*2, activation="relu", name="aa-dec"),
                tf.keras.layers.Dense(X_id.shape[1], name="aa-X"),
            ]
        )
        self.aa_model.compile(optimizer='adam', loss="mse", metrics=["mse", "mae"])
        self.aa_model.fit(
            X_id, X_id,
            validation_data=(X_id_valid, X_id_valid),
            batch_size=200, epochs=200, verbose=1, callbacks=[
                tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=10,
                    restore_best_weights=True,
                )
            ],
        )
        
        print("Fitting covariance")
        
        self.cov = EmpiricalCovariance().fit((self.aa_model.predict(X_id) - X_id))
        
        print("Generate FGSM inputs")
        
        x_id = tf.constant(X_id_valid, dtype=tf.float32)
        x_id = tf.random.normal(mean=x_id, stddev=0.01, shape=x_id.shape)
        
        with tf.GradientTape() as tape:
            tape.watch(x_id)

            x_pred_id = self.aa_model(x_id)
            x_mse_id = (x_pred_id - x_id) ** 2

        adv_grad = tape.gradient(x_mse_id, x_id)

        X_ood = (x_id + tf.math.sign(adv_grad) * tf.math.abs(tf.random.normal(
            mean=2.0, stddev=0.5, shape=[len(X_id_valid), 1],
        ))).numpy()
        
        print("Train classifier")
        
        M_id = self.cov.mahalanobis(self.aa_model.predict(X_id_valid) - X_id_valid)
        M_ood = self.cov.mahalanobis(self.aa_model.predict(X_ood) - X_ood)
        
        self.scaler = StandardScaler().fit(M_id.reshape(-1, 1))
        
        self.model = LogisticRegression(
            penalty='none', class_weight="balanced", random_state=self.rng,
        ).fit(
            np.concatenate([
                self.scaler.transform(M_id.reshape(-1, 1)),
                self.scaler.transform(M_ood.reshape(-1, 1)),
            ], axis=0).reshape(-1, 1),
            np.concatenate([
                np.ones(len(M_id)), np.zeros(len(M_ood)),
            ], axis=0),
        )
        
        return self
    
    def score(self, X_test):
        return self.model.predict_proba(self.scaler.transform(
            self.cov.mahalanobis(self.aa_model.predict(X_test) - X_test).reshape(-1, 1)
        ))[:,1]

In [ ]:
from sklearn.linear_model import LogisticRegression

class AutoAssociativeErrorLogisticDetector(OutOfDistributionDetector):
    def __init__(self, *args, rng=None, **kwargs):
        self.rng = rng
        
    def fit(self, X_id, Y_id, X_id_valid, Y_id_valid):
        print("Fitting PCA")
        
        bn = np.searchsorted(np.cumsum(
            PCA(random_state=self.rng).fit(X_id).explained_variance_ratio_
        ), 0.95)
        
        print(f"Fitting AA with bn={bn}")
        
        tf.random.set_seed(int.from_bytes(self.rng.bytes(4), "little"))
        
        self.aa_model = tf.keras.Sequential(
            [
                tf.keras.layers.InputLayer(input_shape=X_id.shape[1:]),
                tf.keras.layers.Dense(X_id.shape[1]*2, activation="relu", name="aa-enc"),
                tf.keras.layers.Dense(bn, activation="relu", name="aa-bn"),
                tf.keras.layers.Dense(X_id.shape[1]*2, activation="relu", name="aa-dec"),
                tf.keras.layers.Dense(X_id.shape[1], name="aa-X"),
            ]
        )
        self.aa_model.compile(optimizer='adam', loss="mse", metrics=["mse", "mae"])
        self.aa_model.fit(
            X_id, X_id,
            validation_data=(X_id_valid, X_id_valid),
            batch_size=200, epochs=200, verbose=1, callbacks=[
                tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=10,
                    restore_best_weights=True,
                )
            ],
        )
        
        print("Generate FGSM inputs")
        
        x_id = tf.constant(X_id_valid, dtype=tf.float32)
        x_id = tf.random.normal(mean=x_id, stddev=0.01, shape=x_id.shape)
        
        with tf.GradientTape() as tape:
            tape.watch(x_id)

            x_pred_id = self.aa_model(x_id)
            x_mse_id = (x_pred_id - x_id) ** 2

        adv_grad = tape.gradient(x_mse_id, x_id)

        X_ood = (x_id + tf.math.sign(adv_grad) * tf.math.abs(tf.random.normal(
            mean=2.0, stddev=0.5, shape=[len(X_id_valid), 1],
        ))).numpy()
        
        print("Train classifier")
        
        self.model = LogisticRegression(
            penalty='none', class_weight="balanced", random_state=self.rng,
        ).fit(
            np.concatenate([
                np.abs(self.aa_model.predict(X_id_valid) - X_id_valid),
                np.abs(self.aa_model.predict(X_ood) - X_ood),
            ], axis=0),
            np.concatenate([
                np.ones(len(X_id_valid)), np.zeros(len(X_ood)),
            ], axis=0),
        )
        
        return self
    
    def score(self, X_test):
        return self.model.predict_proba(np.abs(self.aa_model.predict(X_test) - X_test))[:,1]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

class TruncatedPCAMahalanobisLogisticDetector(OutOfDistributionDetector):
    def __init__(self, *args, rng=None, **kwargs):
        self.rng = rng
        
    def _predict_truncated_pca(self, X):
        if self.pca.mean_ is not None:
            X = X - self.pca.mean_
        
        X_trans = np.dot(X, self.pca.components_[:self.bn].T)
        X = np.dot(X_trans, self.pca.components_[:self.bn])
        
        if self.pca.mean_ is not None:
            X = X + self.pca.mean_
        
        return X
        
    def fit(self, X_id, Y_id, X_id_valid, Y_id_valid):
        print("Fitting PCA")
        
        self.pca = PCA(random_state=self.rng).fit(X_id)
        self.bn = np.searchsorted(np.cumsum(self.pca.explained_variance_ratio_), 0.95)
        
        print("Fitting covariance")
        
        self.cov = EmpiricalCovariance().fit((self._predict_truncated_pca(X_id) - X_id))
        
        print("Generate FGSM inputs")
        
        adv_grad = self.pca.components_[self.bn]
        
        X_ood = self.rng.normal(loc=X_id_valid, scale=0.01) + np.sign(adv_grad) * np.abs(
            self.rng.normal(loc=2.0, scale=0.5, size=(len(X_id_valid), 1))
        ) * self.rng.choice([-1, 1])
        
        print("Train classifier")
        
        M_id = self.cov.mahalanobis(self._predict_truncated_pca(X_id_valid) - X_id_valid)
        M_ood = self.cov.mahalanobis(self._predict_truncated_pca(X_ood) - X_ood)
        
        self.scaler = StandardScaler().fit(M_id.reshape(-1, 1))
        
        self.model = LogisticRegression(
            penalty='none', class_weight="balanced", random_state=self.rng,
        ).fit(
            np.concatenate([
                self.scaler.transform(M_id.reshape(-1, 1)),
                self.scaler.transform(M_ood.reshape(-1, 1)),
            ], axis=0).reshape(-1, 1),
            np.concatenate([
                np.ones(len(M_id)), np.zeros(len(M_ood)),
            ], axis=0),
        )
        
        print("Finished training the OOD scorer")
        
        return self
    
    def score(self, X_test):
        print("Generating confidence scores ...")
        
        return self.model.predict_proba(self.scaler.transform(
            self.cov.mahalanobis(self._predict_truncated_pca(X_test) - X_test).reshape(-1, 1)
        ))[:,1]

In [ ]:
def train_and_cache_detector(dt: datetime.datetime, clump: float, datasets: dict, models: dict, cls, *args, **kwargs):
    if isinstance(dt, tuple) or isinstance(dt, list):
        dt = tuple(sorted(dt))
    
    model_key = (cls.__name__, dt, clump)
    
    cached = models.get(model_key)
    
    if cached is not None:
        return cached
    
    model_path = f"{cls.__name__.lower()}.score.{hash_for_dt(dt).hexdigest(8)}.{clump}.jl"
    
    if Path(model_path).exists():    
        model = joblib.load(model_path)
        
        models[model_key] = model
        
        return model
    
    dataset = load_and_cache_dataset(dt, clump, datasets)
    
    rng = np.random.RandomState(seed=int.from_bytes(hash_for_dt(dt).digest(4), 'little'))
    
    model = cls(*args, rng=rng, **kwargs).fit(
        X_id=dataset.X_train, Y_id=dataset.Y_train,
        X_id_valid=dataset.X_valid, Y_id_valid=dataset.Y_valid,
    )
    
    joblib.dump(model, model_path)
    
    models[model_key] = model
    
    return model

In [ ]:
dt_train = datetime.datetime(year=2018, month=5, day=15, hour=19)
dt_test = [
    datetime.datetime(year=2018, month=5, day=14, hour=10),
    datetime.datetime(year=2018, month=5, day=17, hour=0),
    datetime.datetime(year=2018, month=5, day=19, hour=4),
    datetime.datetime(year=2018, month=5, day=21, hour=15),
    datetime.datetime(year=2018, month=5, day=23, hour=13),
]

ds_train = load_and_cache_dataset(dt_train, 0.75, DATASETS)
ds_test = load_and_cache_dataset(dt_test, 0.75, DATASETS)

In [ ]:
dt_temp = [
    datetime.datetime(year=2018, month=5, day=15, hour=15),
    datetime.datetime(year=2018, month=5, day=15, hour=17),
    datetime.datetime(year=2018, month=5, day=15, hour=18),
    datetime.datetime(year=2018, month=5, day=15, hour=20),
    datetime.datetime(year=2018, month=5, day=15, hour=21),
    datetime.datetime(year=2018, month=5, day=15, hour=23),
]

ds_temp = load_and_cache_dataset(dt_temp, 0.75, DATASETS)

In [ ]:
from sklearn import metrics
from sklearn.calibration import calibration_curve

In [ ]:
myrng = np.random.RandomState(seed=42)

In [ ]:
cov_id = EmpiricalCovariance().fit(ds_train.X_train)

X_id_cov = myrng.multivariate_normal(
    mean=cov_id.location_, cov=cov_id.covariance_, size=len(ds_train.X_train),
)

In [ ]:
X_ood_cov = []

for f in np.linspace(0.05, 0.95, 10):
    X_ood_cov.append(
        myrng.multivariate_normal(
            mean=cov_id.location_ * f,
            cov=cov_id.covariance_ * f + np.identity(len(cov_id.location_)) * (1-f),
            size=len(ds_train.X_test) // 20,
        )
    )

X_ood_cov = np.concatenate(X_ood_cov, axis=0)

In [ ]:
X_ood_n01 = myrng.normal(loc=0.0, scale=1.0, size=(len(ds_train.X_test) // 2, ds_train.X_train.shape[1]))

In [ ]:
MODELS = dict()

for cls, args, kwargs in [
    (GaussianProcessPercentileDetector, [], dict()),
    (AutoAssociativeMahalanobisPercentileDetector, [], dict()),
    (LogisticClassifierDetector, [], dict()),
    (LogisticTPokeClassifierDetector, [], dict()),
    (GaussianProcessIDWeightDetector, [], dict()),
    (AutoAssociativeMahalanobisIDWeightDetector, [], dict()),
    (GaussianProcessLogisticDetector, [], dict()),
    (AutoAssociativeMahalanobisLogisticDetector, [], dict()),
    (AutoAssociativeErrorLogisticDetector, [], dict()),
    (TruncatedPCAMahalanobisLogisticDetector, [], dict()),
]:      
    detector = train_and_cache_detector(dt_train, 0.75, DATASETS, MODELS, cls, *args, **kwargs)
    
    C_id_train = detector.score(ds_train.X_train)
    C_id_valid = detector.score(ds_train.X_valid)
    C_id_test = detector.score(ds_train.X_test)
    C_id = C_id_test

    C_id_cov = detector.score(X_id_cov)
    C_temp = detector.score(ds_train.X_scaler.transform(
        ds_temp.X_scaler.inverse_transform(np.concatenate([
            ds_temp.X_train, ds_temp.X_valid, ds_temp.X_test,
        ], axis=0))
    ))
    C_trajs = detector.score(ds_train.X_scaler.transform(
        ds_test.X_scaler.inverse_transform(np.concatenate([
            ds_test.X_train, ds_test.X_valid, ds_test.X_test,
        ], axis=0))
    ))
    
    C_ood_cov = detector.score(X_ood_cov)
    C_ood_n01 = detector.score(X_ood_n01)
    C_ood = np.concatenate([C_ood_cov, C_ood_n01], axis=0)
    
    
    fig, ax = plt.subplots(1, 1, figsize=(6, 4))

    ax.set_title(f"{cls.name()}")
    ax.set_xlabel("confidence score $c$")
    ax.set_yticks([])
    ax.set_ylabel("relative frequency")

    ax.set_xlim((-0.05, 1.05))

    # Add some minor random noise to the confidence scores to avoid
    #  singular matrices without affecting the visuals
    v1 = ax.violinplot((
        np.concatenate([
            C_id_test, C_id_valid,
        ], axis=0) + np.random.normal(0.0, 1e-9, size=np.concatenate([
            C_id_test, C_id_valid,
        ], axis=0).shape)
    ).reshape(-1, 1), showextrema=True, vert=False, positions=[7])
    v2 = ax.violinplot((
        C_id_test + np.random.normal(0.0, 1e-9, size=C_id_test.shape)
    ).reshape(-1, 1), showextrema=True, vert=False, positions=[6])
    
    v3 = ax.violinplot((
        C_id_cov + np.random.normal(0.0, 1e-9, size=C_id_cov.shape)
    ).reshape(-1, 1), showextrema=True, vert=False, positions=[5])
    v4 = ax.violinplot((
        C_temp + np.random.normal(0.0, 1e-9, size=C_temp.shape)
    ).reshape(-1, 1), showextrema=True, vert=False, positions=[4])
    v5 = ax.violinplot((
        C_trajs + np.random.normal(0.0, 1e-9, size=C_trajs.shape)
    ).reshape(-1, 1), showextrema=True, vert=False, positions=[3])
    
    v6 = ax.violinplot((
        C_ood_cov + np.random.normal(0.0, 1e-9, size=C_ood_cov.shape)
    ).reshape(-1, 1), showextrema=True, vert=False, positions=[2])
    v7 = ax.violinplot((
        C_ood_n01 + np.random.normal(0.0, 1e-9, size=C_ood_n01.shape)
    ).reshape(-1, 1), showextrema=True, vert=False, positions=[1])

    for v, c in zip([v1, v2, v3, v4, v5, v6, v7], [
        plt.cm.tab20b(1), plt.cm.tab20c(0), 
        plt.cm.tab20c(12), plt.cm.tab20c(8), plt.cm.tab20b(9),
        plt.cm.tab20c(5), plt.cm.tab20c(4),
    ]):
        v["cbars"].set_visible(False)
        v["cbars"].set_color(c)

        s = v["cmins"].get_segments()
        s[0][0,1] -= 0.2
        s[0][1,1] += 0.2
        v["cmins"].set_segments(s)
        v["cmins"].set_linestyle(":")
        v["cmins"].set_color(c)

        s = v["cmaxes"].get_segments()
        s[0][0,1] -= 0.2
        s[0][1,1] += 0.2
        v["cmaxes"].set_segments(s)
        v["cmaxes"].set_linestyle(":")
        v["cmaxes"].set_color(c)

        for b in v["bodies"]:
            b.set_color(c)

    ax.text(
        0.5, 7.0, "ID trajectory training+validation data", c=v1["cbars"].get_color()[0],
        va="center", ha="center", path_effects=outlined,
    )
    ax.text(
        0.5, 6.0, "ID trajectory test data", c=v2["cbars"].get_color()[0],
        va="center", ha="center", path_effects=outlined,
    )
    ax.text(
        0.5, 5.0, r"N$(\mu($ID$), \Sigma($ID$))$", c=v3["cbars"].get_color()[0],
        va="center", ha="center", path_effects=outlined,
    )
    ax.text(
        0.5, 4.0, r"$\pm 1$h, $\pm 2$h, and $\pm 4$h trajectories",
        c=v4["cbars"].get_color()[0], va="center", ha="center", path_effects=outlined,
    )
    ax.text(
        0.5, 3.0, "other five trajectories", c=v5["cbars"].get_color()[0],
        va="center", ha="center", path_effects=outlined,
    )
    ax.text(
        0.5, 2.0, r"OOD N$(\mu($ID$), \Sigma($ID$))$ $\rightarrow$ N$(0, 1)$",
        c=v6["cbars"].get_color()[0], va="center", ha="center", path_effects=outlined,
    )
    ax.text(
        0.5, 1.0, "OOD N$(0, 1)$", c=v7["cbars"].get_color()[0],
        va="center", ha="center", path_effects=outlined,
    )
    
    plt.savefig(f"ood.{cls.__name__.lower()}-distribution.pdf", dpi=100, transparent=True, bbox_inches='tight')
    # plt.show()
    plt.close(fig)


    fpr, tpr, roc_thresholds = metrics.roc_curve(np.concatenate([
        np.ones(shape=len(C_id)), np.zeros(shape=len(C_ood)),
    ]), np.concatenate([
        C_id, C_ood,
    ]))
    roc_auc = metrics.auc(fpr, tpr)

    pr0, rc0, pr_thresholds0 = metrics.precision_recall_curve(np.concatenate([
        np.ones(shape=len(C_id)), np.zeros(shape=len(C_ood)),
    ]), 1-np.concatenate([
        C_id, C_ood,
    ]), pos_label=0)
    pr_auc0 = metrics.auc(rc0, pr0)
    ap0 = metrics.average_precision_score(np.concatenate([
        np.ones(shape=len(C_id)), np.zeros(shape=len(C_ood)),
    ]), 1-np.concatenate([
        C_id, C_ood,
    ]), pos_label=0)

    pr1, rc1, pr_thresholds1 = metrics.precision_recall_curve(np.concatenate([
        np.ones(shape=len(C_id)), np.zeros(shape=len(C_ood)),
    ]), np.concatenate([
        C_id, C_ood,
    ]), pos_label=1)
    pr_auc1 = metrics.auc(rc1, pr1)
    ap1 = metrics.average_precision_score(np.concatenate([
        np.ones(shape=len(C_id)), np.zeros(shape=len(C_ood)),
    ]), np.concatenate([
        C_id, C_ood,
    ]), pos_label=1)

    pid, pc = calibration_curve(
        np.concatenate([
            np.ones(shape=len(C_id)), np.zeros(shape=len(C_ood)),
        ], axis=0),
        np.concatenate([C_id, C_ood], axis=0),
        n_bins=10, strategy="uniform"
    )
    cal_err = np.sqrt(np.mean((pid-pc)**2))


    fig, axs = plt.subplots(2, 2, figsize=(4, 4))

    fig.suptitle("Confidence Score Calibration", y=0.935)

    for ax in axs.flatten():
        ax.set_xlim((-0.05, 1.05))
        ax.set_ylim((-0.05, 1.05))

        ax.set_xticks([0.0, 1.0])
        ax.set_yticks([0.0, 1.0])

    axs[0,0].set_xlabel("mean $c$", labelpad=-10.0)
    axs[0,0].set_ylabel("ID rate", labelpad=-10.0)

    axs[0,1].set_xlabel("fp rate", labelpad=-10.0)
    axs[0,1].set_ylabel("tp rate", labelpad=-10.0)

    axs[1,0].set_xlabel("recall", labelpad=-10.0)
    axs[1,0].set_ylabel("precision", labelpad=-10.0)

    axs[1,1].set_xlabel("recall", labelpad=-10.0)
    axs[1,1].set_ylabel("precision", labelpad=-10.0)

    axs[0,0].plot([0, 1], [0, 1], "k:")
    axs[0,0].plot(pc, pid, "s-", c="black")
    axs[0,0].legend(handles=[
        mpl.patches.Patch(label=f"RMSCE = {cal_err:.2}")
    ], loc="lower right", handlelength=0, handletextpad=0)

    axs[0,1].plot(fpr, tpr, c=plt.cm.tab20(4))
    axs[0,1].legend(handles=[
        mpl.patches.Patch(label=f"ROC-AUC = {roc_auc:.2}")
    ], loc="lower right", handlelength=0, handletextpad=0)

    axs[1,0].plot(rc1, pr1, c=plt.cm.tab20(0))
    axs[1,0].legend(handles=[
        mpl.patches.Patch(label=f"ID-Detection:\nPR-AUC = {pr_auc1:.2}\nAP = {ap1:.2}")
    ], loc="lower left", handlelength=0, handletextpad=0)

    axs[1,1].plot(rc0, pr0, c=plt.cm.tab20(6))
    axs[1,1].legend(handles=[
        mpl.patches.Patch(label=f"OOD-Detection:\nPR-AUC = {pr_auc0:.2}\nAP = {ap0:.2}")
    ], loc="lower left", handlelength=0, handletextpad=0)

    plt.savefig(f"ood.{cls.__name__.lower()}-calibration.pdf", dpi=100, transparent=True, bbox_inches='tight')
    # plt.show()
    plt.close(fig)